# Forest Sites Interactive Map Viewer

Interactive Leaflet maps for examining forest site locations defined in `forest_sites.yaml`.

**Purpose:** Verify site accuracy, examine forest structure, and confirm 3DEP lidar coverage.

## Features
- Interactive maps with satellite basemap
- Site bounding boxes shown as polygons (green)
- **3DEP lidar flight boundaries overlay (cyan)** - shows available lidar coverage
- Pop-up information for each site
- All 18 forest sites across the US

In [ ]:
# Install ipyleaflet if not available
try:
    import ipyleaflet
except ImportError:
    import subprocess
    subprocess.check_call(['pip', 'install', 'ipyleaflet'])
    import ipyleaflet

import yaml
import json
import requests
from pathlib import Path
from shapely.geometry import shape, box
from ipyleaflet import Map, GeoJSON, basemaps, basemap_to_tiles, LayersControl, FullScreenControl, ScaleControl
from ipywidgets import HTML, Layout

print(f"ipyleaflet version: {ipyleaflet.__version__}")

In [ ]:
# Load forest sites configuration
sites_yaml_path = Path('../forest_sites.yaml')
geojson_dir = Path('../geojson')

with open(sites_yaml_path, 'r') as f:
    config = yaml.safe_load(f)

sites = config['sites']
print(f"Loaded {len(sites)} forest sites")

# Load combined GeoJSON
with open(geojson_dir / 'all_sites.geojson', 'r') as f:
    all_sites_geojson = json.load(f)

print(f"GeoJSON features: {len(all_sites_geojson['features'])}")

# Fetch 3DEP lidar boundaries from Hobu Inc
print("\nFetching 3DEP lidar boundaries...")
lidar_url = 'https://raw.githubusercontent.com/hobuinc/usgs-lidar/master/boundaries/resources.geojson'
response = requests.get(lidar_url)
lidar_boundaries = response.json()
print(f"Loaded {len(lidar_boundaries['features'])} 3DEP lidar datasets")

# Calculate total point count
total_points = sum(f['properties'].get('count', 0) for f in lidar_boundaries['features'])
print(f"Total points available: {total_points:,.0f}")

In [ ]:
# Check lidar coverage for each site
def check_lidar_coverage(site_bbox, lidar_geojson):
    """
    Check which lidar datasets intersect with a site's bounding box.
    
    Args:
        site_bbox: [west, south, east, north]
        lidar_geojson: GeoJSON FeatureCollection of lidar boundaries
        
    Returns:
        List of intersecting dataset names and total point estimate
    """
    west, south, east, north = site_bbox
    site_box = box(west, south, east, north)
    
    intersecting = []
    for feature in lidar_geojson['features']:
        try:
            lidar_geom = shape(feature['geometry'])
            if site_box.intersects(lidar_geom):
                intersecting.append({
                    'name': feature['properties'].get('name', 'Unknown'),
                    'count': feature['properties'].get('count', 0)
                })
        except Exception:
            continue
    
    return intersecting

# Check coverage for all sites
print("Checking 3DEP lidar coverage for each site...")
print("-" * 80)

site_coverage = {}
for site_id, site_data in sites.items():
    coverage = check_lidar_coverage(site_data['bbox'], lidar_boundaries)
    site_coverage[site_id] = coverage
    
    if coverage:
        total_points = sum(c['count'] for c in coverage)
        print(f"✓ {site_id:<30} {len(coverage):>2} dataset(s), ~{total_points:>15,} points")
    else:
        print(f"✗ {site_id:<30} NO LIDAR COVERAGE")

sites_with_coverage = sum(1 for c in site_coverage.values() if c)
print("-" * 80)
print(f"Sites with lidar coverage: {sites_with_coverage}/{len(sites)}")

## Overview Map - All Sites

This map shows all forest sites across the continental US with 3DEP lidar coverage overlay.

**Legend:**
- **Red polygons**: Forest site AOIs
- **Cyan polygons**: 3DEP lidar flight boundaries (toggle in layer control)

In [ ]:
# Create overview map centered on continental US
overview_map = Map(
    center=(39.0, -98.0),
    zoom=4,
    layout=Layout(width='100%', height='600px'),
    scroll_wheel_zoom=True
)

# Add satellite basemap
satellite_layer = basemap_to_tiles(basemaps.Esri.WorldImagery)
overview_map.add(satellite_layer)

# Add 3DEP lidar boundaries layer (cyan, semi-transparent)
lidar_layer = GeoJSON(
    data=lidar_boundaries,
    style={
        'color': '#00ffff',
        'weight': 1,
        'fillColor': '#00ffff',
        'fillOpacity': 0.15
    },
    hover_style={'fillOpacity': 0.3},
    name='3DEP Lidar Coverage'
)
overview_map.add(lidar_layer)

# Add all forest sites as GeoJSON layer (red)
sites_layer = GeoJSON(
    data=all_sites_geojson,
    style={'color': '#ff0000', 'weight': 3, 'fillColor': '#ff0000', 'fillOpacity': 0.3},
    hover_style={'fillOpacity': 0.6},
    name='Forest Sites'
)
overview_map.add(sites_layer)

# Add controls
overview_map.add(FullScreenControl())
overview_map.add(ScaleControl(position='bottomleft'))
overview_map.add(LayersControl(position='topright'))

print("Overview Map - All Forest Sites with 3DEP Lidar Coverage")
print("Use the layer control (top right) to toggle lidar boundaries")
overview_map

## Site Summary Table

In [ ]:
# Display site summary
print(f"{'Site ID':<30} {'Name':<35} {'State':<5} {'Priority':<8} {'Max Height':<10}")
print("-" * 95)

for site_id, site_data in sorted(sites.items(), key=lambda x: (x[1]['priority'], x[0])):
    print(f"{site_id:<30} {site_data['name']:<35} {site_data['state']:<5} {site_data['priority']:<8} {site_data['expected_max_height_m']:<10}m")

---

## Individual Site Maps

Below are detailed maps for each forest site, organized by region. Use these to examine forest structure and verify location accuracy.

In [ ]:
def get_intersecting_lidar(site_bbox, lidar_geojson):
    """Get lidar features that intersect with site bbox."""
    west, south, east, north = site_bbox
    site_box = box(west, south, east, north)
    
    intersecting_features = []
    for feature in lidar_geojson['features']:
        try:
            lidar_geom = shape(feature['geometry'])
            if site_box.intersects(lidar_geom):
                intersecting_features.append(feature)
        except Exception:
            continue
    
    return {
        "type": "FeatureCollection",
        "features": intersecting_features
    }

def create_site_map(site_id, site_data, height='400px'):
    """
    Create an interactive map for a single forest site with lidar coverage.
    
    Args:
        site_id: The site identifier
        site_data: Dictionary with site configuration
        height: Map height (default 400px)
        
    Returns:
        ipyleaflet.Map object
    """
    bbox = site_data['bbox']  # [west, south, east, north]
    west, south, east, north = bbox
    
    # Calculate center
    center_lat = (south + north) / 2
    center_lon = (west + east) / 2
    
    # Create map
    m = Map(
        center=(center_lat, center_lon),
        zoom=13,
        layout=Layout(width='100%', height=height),
        scroll_wheel_zoom=True
    )
    
    # Add satellite basemap
    satellite = basemap_to_tiles(basemaps.Esri.WorldImagery)
    m.add(satellite)
    
    # Get intersecting lidar boundaries for this site
    site_lidar = get_intersecting_lidar(bbox, lidar_boundaries)
    
    # Add lidar coverage layer (cyan)
    if site_lidar['features']:
        lidar_layer = GeoJSON(
            data=site_lidar,
            style={
                'color': '#00ffff',
                'weight': 2,
                'fillColor': '#00ffff',
                'fillOpacity': 0.1
            },
            hover_style={'fillOpacity': 0.25},
            name='3DEP Lidar'
        )
        m.add(lidar_layer)
    
    # Create polygon from bbox
    polygon_coords = [
        [west, south],
        [east, south],
        [east, north],
        [west, north],
        [west, south]
    ]
    
    geojson_data = {
        "type": "FeatureCollection",
        "features": [{
            "type": "Feature",
            "properties": {
                "name": site_data['name'],
                "description": site_data['description']
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [polygon_coords]
            }
        }]
    }
    
    # Add site polygon layer (green)
    geo_layer = GeoJSON(
        data=geojson_data,
        style={
            'color': '#00ff00',
            'weight': 3,
            'fillColor': '#00ff00',
            'fillOpacity': 0.1
        },
        hover_style={'fillOpacity': 0.3},
        name='Site AOI'
    )
    m.add(geo_layer)
    
    # Add controls
    m.add(FullScreenControl())
    m.add(ScaleControl(position='bottomleft'))
    m.add(LayersControl(position='topright'))
    
    return m

### Pacific Northwest - Temperate Rainforests

In [ ]:
site_id = 'sequoia_giant_forest'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'redwood_humboldt'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'olympic_hoh'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'tongass_mendenhall'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'wind_river'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

### Rocky Mountains - Conifer Forests

In [ ]:
site_id = 'yellowstone_lodgepole'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'rocky_mountain_subalpine'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

### Southwest - Ponderosa Pine

In [ ]:
site_id = 'coconino_ponderosa'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'gila_mixed_conifer'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

### Eastern Deciduous - Hardwood Forests

In [ ]:
site_id = 'great_smoky_cove'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'mark_twain_ozark'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'white_mountain_northern'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

### Boreal/Northern - Mixed Forests

In [ ]:
site_id = 'boundary_waters'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'superior_old_growth'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

### Southeastern - Pine Forests

In [ ]:
site_id = 'ocala_longleaf'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'conecuh_longleaf'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

### Special Research Sites

In [ ]:
site_id = 'harvard_forest'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'niwot_ridge'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

---

## Notes on Site Verification

When examining each site, look for:

1. **Lidar Coverage**: Does the cyan overlay show 3DEP lidar data is available?
2. **Forest Coverage**: Is the AOI actually covered by forest?
3. **Forest Type**: Does the visible canopy match the expected forest type?
4. **Access**: Are there roads or trails visible that might affect the analysis?
5. **Disturbance**: Are there signs of logging, fire, or other disturbances?
6. **Boundary Accuracy**: Does the bounding box capture the intended forest area?

## Map Legend

- **Green polygon**: Site Area of Interest (AOI)
- **Cyan polygon**: 3DEP lidar flight boundary (indicates available data)

## GeoJSON Files

Individual GeoJSON files for each site are available in the `../geojson/` directory:
- `all_sites.geojson` - Combined file with all sites
- `<site_id>.geojson` - Individual site files

## 3DEP Lidar Data Source

Lidar boundaries from: https://github.com/hobuinc/usgs-lidar

In [ ]:
# List generated GeoJSON files
import os

geojson_files = sorted(geojson_dir.glob('*.geojson'))
print(f"Generated GeoJSON files ({len(geojson_files)} total):")
for f in geojson_files:
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name:<40} ({size_kb:.1f} KB)")